# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [1]:
from utility import * 

Here we use $LiH$ as an example for finding QWC fragments. Notice below that each fragment has the same terms on all qubits.  

To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [11]:
lih = get_qubit_hamiltonian(mol='lih', geometry=1, basis='sto-3g', qubit_transf='jw')

qwc_list = get_qwc_group(lih)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
-0.0032403902911914565 [X0 Z1 Z2 Y3 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
0.06558452315458399 [Z6 Z8] +
0.0698018080330067 [Z6 Z9] +
0.0698018080330067 [Z7 Z8] +
0.06558452315458399 [Z7 Z9]

Fragments 2:
-0.0009084689845792959 [Y0 Z1 Z2 Z3 Z4 Z5 Y6 Y7 Z8 Z9 Z10 Y11] +
0.0990222572034092 [Z1 Z8] +
0.09654955103502393 [Z1 Z9] +
0.06832924266504996 [Z2 Z8] +
0.0751247693474757 [Z2 Z9] +
0.0751247693474757 [Z3 Z8] +
0.06832924266504996 [Z3 Z9] +
0.06009702977667725 [Z4 Z8] +
0.07068762352443508 [Z4 Z9] +
0.07068762352443508 [Z5 Z8] +
0.06009702977667725 [Z5 Z9] +
-0.2167542932381841 [Z8] +
0.0782363777898521 [Z8 Z9] +
0.0642851991772659 [Z8 Z10] +
-0.21675429323818407 [Z9] +
0.068118185609469 [Z9 Z10]

Number of fragments: 154


By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of $LiH$ by partitioning its terms into mutually commuting fragments. 

In [12]:
comm_groups = get_commuting_group(lih)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Number of mutually commuting fragments: 36
The first commuting group
-3.93444195675791 [] +
0.007923321157850723 [X0 Y1 Y2 X3] +
-0.00015507836015035223 [X0 Z1 Z2 X3 Y4 Y5] +
-0.002077886902662328 [X0 Z1 Z2 X3 Y6 Y7] +
-0.0020778869026623274 [X0 Z1 Z2 X3 Y8 Y9] +
-0.0021028659073762156 [X0 Z1 Z2 X3 Y10 Y11] +
0.007923321157850723 [Y0 X1 X2 Y3] +
-0.00015507836015035223 [Y0 Z1 Z2 Y3 X4 X5] +
-0.002077886902662328 [Y0 Z1 Z2 Y3 X6 X7] +
-0.0020778869026623274 [Y0 Z1 Z2 Y3 X8 X9] +
-0.0021028659073762156 [Y0 Z1 Z2 Y3 X10 X11] +
0.11709373561826503 [Z0 Z3] +
-0.0001550783601503522 [X1 X2 X4 X5] +
-0.002077886902662328 [X1 X2 X6 X7] +
-0.0020778869026623274 [X1 X2 X8 X9] +
-0.0021028659073762156 [X1 X2 X10 X11] +
-0.0001550783601503522 [Y1 Y2 Y4 Y5] +
-0.002077886902662328 [Y1 Y2 Y6 Y7] +
-0.0020778869026623274 [Y1 Y2 Y8 Y9] +
-0.0021028659073762156 [Y1 Y2 Y10 Y11] +
0.11709373561826503 [Z1 Z2] +
-0.010590593747757844 [X4 X5 Y6 Y7] +
-0.010590593747757839 [X4 X5 Y8 Y9] +
-0.00673303078088139

To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

In [19]:
uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
%time I_ = uqwc * uqwc
print(I_)

This is unitary, U * U^+ = I 
CPU times: user 2min 39s, sys: 0 ns, total: 2min 39s
Wall time: 2min 39s
(0.9999999999999999+0j) []


Applying this unitary gives the qubit-wise commuting form of the first mutually commuting group

In [21]:
#This takes too long and never finishes
%time qwc = uqwc * comm_groups[1]
%time qwc = qwc * uqwc
%time qwc = remove_complex(qwc)

CPU times: user 1.64 s, sys: 32 ms, total: 1.67 s
Wall time: 1.67 s


KeyboardInterrupt: 

CPU times: user 3.04 s, sys: 2 µs, total: 3.04 s
Wall time: 3.04 s


In addition, current quantum computer can measure only the $z$ operators. Thus, QWC fragments with $x$ or $y$ operators require extra single-qubit unitaries that rotate them into $z$.  

In [ ]:
uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

## H4


In [8]:
h4 = get_qubit_hamiltonian(mol='h4', geometry=90, basis='sto-3g', qubit_transf='jw')

qwc_list = get_qwc_group(h4)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

comm_groups = get_commuting_group(h4)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
print(uqwc * uqwc)

qwc = remove_complex(uqwc * comm_groups[1] * uqwc)
print(qwc)


uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Fragments 1: 
-9.87643820460956e-07 [X0 Z1 Z2 Z3 X4 Y5 Z6 Y7]

Fragments 2:
9.721079259864158e-07 [Y0 Z1 Y2 Y3 Z4 Z5 Z6 Y7]

Number of fragments: 57
Number of mutually commuting fragments: 9
The first commuting group
-0.6427167034502642 [] +
1.4124085973531153e-07 [X0 X1 X3 X4] +
1.4124085973531153e-07 [X0 Y1 Y3 X4] +
-9.87643820460956e-07 [X0 Z1 Z2 Z3 X4 X5 Z6 X7] +
-9.87643820460956e-07 [X0 Z1 Z2 Z3 X4 Y5 Z6 Y7] +
-3.718844968007666e-07 [X0 Z1 Z2 Z3 Z5 X6] +
0.03382002546943696 [Y0 X1 X2 Y3] +
-0.03382002546943696 [Y0 Y1 X2 X3] +
0.034268472271777504 [Y0 Z1 Y2 X5 Z6 X7] +
0.034268472271777504 [Y0 Z1 Y2 Y5 Z6 Y7] +
3.6883735565415576e-07 [Y0 Z1 Z3 Z4 Z5 Y6] +
-4.865631750684916e-08 [Z0 X2 Z3 X4] +
-9.721079259864158e-07 [X1 X2 Y3 Z4 Z5 Y6] +
0.034268472271777504 [X1 Z2 X3 X4 Z5 X6] +
0.012952358025868181 [X1 Z2 X3 X5 Z6 X7] +
9.721079259864158e-07 [Y1 X2 X3 Z4 Z5 Y6] +
0.034268472271777504 [Y1 Z2 Y3 X4 Z5 X6] +
0.012952358025868181 [Y1 Z2 Y3 Y5 Z6 Y7] +
0.08362706285461705 [Z1 Z3] +
4